In [1]:
import numpy as np
import os
import cv2
import moviepy.editor as mpy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [10]:
# Theoreical test
pts2 = np.float32([[0, 0], [256, 0], [0, 256], [256, 256]])
pts_m = np.float32([[255.2238, 129.5932], [316.7763, 129.5932], [255.2238 ,191.1457], [316.7763, 191.1457]])

matrix = cv2.getPerspectiveTransform(pts_m.astype(np.float32), pts2.astype(np.float32))
test = matrix @ np.array([[316.7763,129.5932,1]]).T
matrix

array([[ 4.15905085e+00,  0.00000000e+00, -1.06148876e+03],
       [ 0.00000000e+00,  4.15905085e+00, -5.38984711e+02],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00]])

$$
\begin{array}{c}
{\left[\begin{array}{c}
t_{i} x_{i}^{\prime} \\
t_{i} y_{i}^{\prime} \\
t_{i}
\end{array}\right]=\operatorname{map}_{-} \operatorname{matrix} \cdot\left[\begin{array}{c}
x_{i} \\
y_{i} \\
1
\end{array}\right]} \\
\operatorname{dst}(i)=\left(x_{i}^{\prime}, y_{i}^{\prime}\right), \operatorname{src}(i)=\left(x_{i}, y_{i}\right), i=0,1,2,3
\end{array}
$$

In [14]:
p1 = np.array([[256, 0,1]])
p2 = np.array([[316.7763,129.5932,1]])

In [29]:
"""
/* Calculates coefficients of perspective transformation
 * which maps (xi,yi) to (ui,vi), (i=1,2,3,4):
 *
 *      c00*xi + c01*yi + c02
 * ui = ---------------------
 *      c20*xi + c21*yi + c22
 *
 *      c10*xi + c11*yi + c12
 * vi = ---------------------
 *      c20*xi + c21*yi + c22
 *
 * Coefficients are calculated by solving linear system:
 * / x0 y0  1  0  0  0 -x0*u0 -y0*u0 \ /c00\ /u0\
 * | x1 y1  1  0  0  0 -x1*u1 -y1*u1 | |c01| |u1|
 * | x2 y2  1  0  0  0 -x2*u2 -y2*u2 | |c02| |u2|
 * | x3 y3  1  0  0  0 -x3*u3 -y3*u3 |.|c10|=|u3|,
 * |  0  0  0 x0 y0  1 -x0*v0 -y0*v0 | |c11| |v0|
 * |  0  0  0 x1 y1  1 -x1*v1 -y1*v1 | |c12| |v1|
 * |  0  0  0 x2 y2  1 -x2*v2 -y2*v2 | |c20| |v2|
 * \  0  0  0 x3 y3  1 -x3*v3 -y3*v3 / \c21/ \v3/
 *
 * where:
 *   cij - matrix coefficients, c22 = 1
 */

c00*xi + c01*yi + c02 - c20*xi*ui - c21*yi*ui = ui
c10*xi + c11*yi + c12 - c20*xi*vi - c21*yi*vi = vi
"""
x = np.array([[0,1,2,3],[4,5,6,7]])
b= np.array([x.flatten()])


array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7]])

In [44]:
def findPerspectiveMatrix(x = np.zeros([4,2]), u = np.zeros([4,2])):
    c = np.zeros([3,3])
    c[2][2] = 1
    x = x.T
    u = u.T

    # Holy Matrix

    A = np.array([
        [x[0][0], x[1][0], 1, 0, 0, 0, -x[0][0]*u[0][0], -x[1][0]*u[0][0]],
        [x[0][1], x[1][1], 1, 0, 0, 0, -x[0][1]*u[0][1], -x[1][1]*u[0][1]],
        [x[0][2], x[1][2], 1, 0, 0, 0, -x[0][2]*u[0][2], -x[1][2]*u[0][2]],
        [x[0][3], x[1][3], 1, 0, 0, 0, -x[0][3]*u[0][3], -x[1][3]*u[0][3]],
        [0, 0, 0, x[0][0], x[1][0], 1, -x[0][0]*u[1][0], -x[1][0]*u[1][0]],
        [0, 0, 0, x[0][1], x[1][1], 1, -x[0][1]*u[1][1], -x[1][1]*u[1][1]],
        [0, 0, 0, x[0][2], x[1][2], 1, -x[0][2]*u[1][2], -x[1][2]*u[1][2]],
        [0, 0, 0, x[0][3], x[1][3], 1, -x[0][3]*u[1][3], -x[1][3]*u[1][3]],
    ])
    
    b= np.array([u.flatten()])
    return np.append(np.linalg.pinv(A) @ b.T,1).reshape(3,3)

findPerspectiveMatrix(pts_m,pts2)


array([[ 4.15905085e+00,  5.34731916e-12, -1.06148876e+03],
       [ 7.51137373e-13,  4.15905085e+00, -5.38984711e+02],
       [ 4.23827640e-14,  1.06579750e-14,  1.00000000e+00]])

In [35]:
matrix

array([[ 4.15905085e+00,  0.00000000e+00, -1.06148876e+03],
       [ 0.00000000e+00,  4.15905085e+00, -5.38984711e+02],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00]])

In [103]:
# This function reads the planes file and returns result as matrix
def readFile(address):
    f = open(address, "r")
    s = f.read()
    s = s.split("\n")
    s = s[:-1]
    s_array = []
    for x in s:
        splited = x.split(")(")
        for i in range(0,len(splited)):
            splited[i] = splited[i].replace("(","")
            splited[i] = splited[i].replace(")","")
            splited[i] = np.array(splited[i].split(" ")).astype(np.float64)
        s_array.append(np.array(splited).astype(np.float64))

    return np.array(s_array).astype(np.float64)

# This function reads the planes by their adress
def readAllPlanes(number_of_planes, plane_name = "Plane_", file_format=".txt"):
    planes = []
    for i in range(1,number_of_planes+1):
        planes.append(readFile(plane_name + str(i) + file_format))
    return np.array(planes).astype(np.float64)

planes = readAllPlanes(9)

In [104]:
i=2
np.mean(planes[0][0].T[:3].T)
depthPlaneMatrix = np.zeros([9])
for k in range(0,planes.shape[0]):
    depthPlaneMatrix[k] = np.mean(planes[k][i].T[:3].T)

depthPlaneMatrix = depthPlaneMatrix.tolist()

In [118]:
planes = ["ahmet","dfds","fds2","kahmet","alım","mesut","yedi","sekiz","dokuz"]
for i in range(0,len(planes)):
    planes[i] = [planes[i],depthPlaneMatrix[i]]


def sortPlane(plane):
    return plane[1]

planes.sort(key=sortPlane)
for i in range(0, len(planes)):
    planes[i] = planes[i][0]

planes

['fds2', 'kahmet', 'dfds', 'alım', 'dokuz', 'mesut', 'ahmet', 'yedi', 'sekiz']